# Import

In [82]:
!pip install pymorphy2

In [1]:
import os, re, datetime, sys, getopt, fnmatch, time, codecs
from os.path import exists
import pymorphy2


# Sorting UA/RU/RUA/None per folders

In [2]:
morph = pymorphy2.MorphAnalyzer('/Users/vvs/Documents/WORK/WV/opcorpora_ua/uadict')

In [3]:
from gensim import corpora
from pprint import pprint
from gensim.utils import simple_preprocess
from smart_open import smart_open

In [4]:
class ReadTxtFiles(object):
    def __init__(self, dirname):
        self.dirname = dirname
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname), encoding='utf-8'):
                yield simple_preprocess(line)

In [5]:
dictionary = corpora.Dictionary(ReadTxtFiles("/Users/vvs/Documents/WORK/ИПСА/TA-2020/coronavirus/process_1587071607"))

In [6]:
dictionary.token2id

{'влада': 0,
 'відеозвернення': 1,
 'віталій': 2,
 'громадське': 3,
 'запроваджуймо': 4,
 'київ': 5,
 'кличка': 6,
 'користування': 7,
 'мер': 8,
 'обмеження': 9,
 'повідоммо': 10,
 'понеділок': 11,
 'сувора': 12,
 'транспорт': 13,
 'багато': 14,
 'досі': 15,
 'живімо': 16,
 'зрозуміймо': 17,
 'людина': 18,
 'ма': 19,
 'не': 20,
 'нові': 21,
 'реалія': 22,
 'які': 23,
 'всі': 24,
 'віна': 25,
 'далеко': 26,
 'дотримуймося': 27,
 'заходи': 28,
 'кияни': 29,
 'обмежувальні': 30,
 'пояснімо': 31,
 'підприємство': 32,
 'рішення': 33,
 'так': 34,
 'ці': 35,
 'березень': 36,
 'важливі': 37,
 'виключно': 38,
 'громадські': 39,
 'життя': 40,
 'забезпечуймо': 41,
 'критично': 42,
 'пасажир': 43,
 'перевозьмо': 44,
 'робот': 45,
 'столиця': 46,
 'сфера': 47,
 'ті': 48,
 'їдьмо': 49,
 'безкоштовно': 50,
 'возімо': 51,
 'карантин': 52,
 'лікар': 53,
 'таксі': 54,
 'аваков': 55,
 'захворіймо': 56,
 'коронавірус': 57,
 'священник': 58,
 'тотальні': 59,
 'україна': 60,
 'хотім': 61,
 'хроніка': 62,
 

In [14]:
from gensim.models import Word2Vec
from multiprocessing import cpu_count

In [25]:
txts3 = [i for i in ReadTxtFiles("/Users/vvs/Documents/WORK/ИПСА/TA-2020/coronavirus/process_1587071607") if i]
print(txts3[0:5])

[['влада', 'київ', 'понеділок', 'запроваджуймо', 'сувора', 'обмеження', 'користування', 'громадське', 'транспорт', 'повідоммо', 'мер', 'віталій', 'кличка', 'відеозвернення'], ['ма', 'живімо', 'нові', 'реалія', 'які', 'багато', 'людина', 'досі', 'не', 'зрозуміймо'], ['обмежувальні', 'заходи', 'дотримуймося', 'далеко', 'не', 'всі', 'кияни', 'підприємство', 'так', 'віна', 'пояснімо', 'ці', 'рішення'], ['понеділок', 'березень', 'громадські', 'транспорт', 'столиця', 'перевозьмо', 'виключно', 'ті', 'пасажир', 'які', 'їдьмо', 'робот', 'підприємство', 'забезпечуймо', 'критично', 'важливі', 'сфера', 'життя'], ['таксі', 'возімо', 'лікар', 'безкоштовно', 'карантин']]


In [234]:
from gensim.models import phrases
bigram = phrases.Phrases(txts3, min_count=3, threshold=3)
print(txts3[5], bigram[txts3[5]])

['хроніка', 'коронавірус', 'аваков', 'хотім', 'тотальні', 'карантин', 'україна', 'захворіймо', 'священник'] ['хроніка_коронавірус', 'аваков', 'хотім', 'тотальні', 'карантин_україна', 'захворіймо', 'священник']


In [239]:
bitxts = [bigram[line] for line in txts3]
model4 = Word2Vec(bitxts, min_count=10)

In [254]:
model4.wv.most_similar('маска')

[('країна', 0.9998997449874878),
 ('ма', 0.9998976588249207),
 ('уряд', 0.9998863935470581),
 ('була', 0.9998846650123596),
 ('робот', 0.9998791217803955),
 ('кажімо', 0.9998771548271179),
 ('маймо', 0.9998756647109985),
 ('вірус', 0.9998753070831299),
 ('навіть', 0.9998750686645508),
 ('ситуація', 0.9998749494552612)]

In [257]:
model4.wv.most_similar('карантин_україна')

[('дитина', 0.999315619468689),
 ('китай', 0.9993109703063965),
 ('росія', 0.9993091225624084),
 ('наприклад', 0.9993081092834473),
 ('світ', 0.9992881417274475),
 ('намагаймося', 0.9992845058441162),
 ('розповім', 0.9992834329605103),
 ('випадок', 0.9992806315422058),
 ('кількість', 0.9992763996124268),
 ('того', 0.9992757439613342)]

In [259]:
model4.wv.most_similar('хроніка_коронавірус')

[('поки', 0.9987032413482666),
 ('туберкульоз', 0.9986830949783325),
 ('було', 0.9986785650253296),
 ('додаваймо', 0.9986751675605774),
 ('заявімо', 0.9986695051193237),
 ('підімо', 0.998668909072876),
 ('тепер', 0.9986592531204224),
 ('кордон', 0.9986591339111328),
 ('саме', 0.9986487627029419),
 ('часто', 0.9986477494239807)]

In [26]:
txt5 = [i for i in ReadTxtFiles("/Users/vvs/Documents/WORK/ИПСА/TA-2020/coronavirus/process_1587080166") if i]
print(txt5[0:5])
model5 = Word2Vec(txt5, min_count=30, size=300, iter=50, window=22)

[['влада', 'київ', 'понеділок', 'запроваджуймо', 'сувора', 'обмеження', 'користування', 'громадське', 'транспорт', 'повідоммо', 'мер', 'віталій', 'кличка', 'відеозвернення'], ['ма', 'живімо', 'нові', 'реалія', 'багато', 'людина', 'досі', 'зрозуміймо'], ['обмежувальні', 'заходи', 'дотримуймося', 'далеко', 'кияни', 'підприємство', 'віна', 'пояснімо', 'рішення'], ['понеділок', 'березень', 'громадські', 'транспорт', 'столиця', 'перевозьмо', 'виключно', 'пасажир', 'їдьмо', 'робот', 'підприємство', 'забезпечуймо', 'критично', 'важливі', 'сфера', 'життя'], ['таксі', 'возімо', 'лікар', 'безкоштовно', 'карантин']]


In [27]:
model5.wv.most_similar('маска')

[('захищаймо', 0.6689644455909729),
 ('обов', 0.521673858165741),
 ('думка', 0.44577938318252563),
 ('товар', 0.4228026568889618),
 ('симптом', 0.4073348641395569),
 ('церква', 0.39973947405815125),
 ('аптека', 0.3898394703865051),
 ('спосіб', 0.38760966062545776),
 ('пасажир', 0.38755255937576294),
 ('більше', 0.3841989040374756)]

In [ ]:
# [('захищаймо', 0.6258448958396912),
#  ('антисептик', 0.5931494235992432),
#  ('захистити', 0.5645320415496826),
#  ('захисні', 0.5603119134902954),
#  ('носій', 0.5470014810562134),
#  ('костюм', 0.5231153964996338),
#  ('вважаймося', 0.5121830105781555),
#  ('язкові', 0.48897719383239746),
#  ('норма', 0.48588085174560547),
#  ('громадські', 0.47002851963043213)]
# Для сравнения первый запуск без настроек

In [333]:
bigram2 = phrases.Phrases(txt5, min_count=3, threshold=22)
bitxts2 = [bigram2[line] for line in txt5]
print(bitxts[0:20])

[['влада', 'київ', 'понеділок', 'запроваджуймо', 'сувора', 'обмеження', 'користування', 'громадське_транспорт', 'повідоммо', 'мер', 'віталій_кличка', 'відеозвернення'], ['ма_живімо', 'нові', 'реалія', 'багато', 'людина', 'досі', 'зрозуміймо'], ['обмежувальні_заходи', 'дотримуймося', 'далеко', 'кияни', 'підприємство', 'віна', 'пояснімо', 'рішення'], ['понеділок', 'березень', 'громадські_транспорт', 'столиця', 'перевозьмо', 'виключно', 'пасажир', 'їдьмо_робот', 'підприємство', 'забезпечуймо', 'критично', 'важливі', 'сфера', 'життя'], ['таксі_возімо', 'лікар_безкоштовно', 'карантин'], ['хроніка_коронавірус', 'аваков', 'хотім', 'тотальні', 'карантин', 'україна', 'захворіймо', 'священник'], ['зможімо', 'іздити', 'транспорт'], ['слово', 'віталій_кличка', 'медичні_працівник', 'рятувальник', 'правоохоронець', 'фахівець', 'сфера', 'працівник', 'продуктові_магазин', 'аптека', 'співробітник', 'стратегічні', 'підприємство'], ['перевезення', 'будьмо', 'розроблені', 'спеціальні', 'маршрут', 'громадс

In [334]:
model6 = Word2Vec(bitxts2, min_count=30, size=300, iter=50, window=22)

In [335]:
model6.wv.most_similar('маска')

[('аптека', 0.6203588247299194),
 ('засіб', 0.5809340476989746),
 ('захищаймо_медичні', 0.5781702995300293),
 ('маска_вірус', 0.5238064527511597),
 ('товар', 0.5036200284957886),
 ('супермаркет', 0.4926426410675049),
 ('великі', 0.4654087424278259),
 ('ринка', 0.46487897634506226),
 ('магазин', 0.4629417955875397),
 ('рука', 0.44699254631996155)]

In [336]:
model6.wv.most_similar('захворювання')

[('інфекція', 0.7220960855484009),
 ('смерть', 0.7193725109100342),
 ('зараження', 0.7098100185394287),
 ('інфіковані', 0.6770030856132507),
 ('коронавірус', 0.6726216077804565),
 ('стан_березень', 0.6338338851928711),
 ('тестування', 0.6336660385131836),
 ('вірус', 0.6335515975952148),
 ('випадок', 0.6324660181999207),
 ('хвороба', 0.6273006200790405)]

In [340]:
model6.wv.most_similar('транспорт')

[('обмеження', 0.6445250511169434),
 ('режим', 0.5543088912963867),
 ('часом', 0.5538583397865295),
 ('поліція', 0.5514323711395264),
 ('робот', 0.5496370196342468),
 ('працюймо', 0.5450451970100403),
 ('послуга', 0.5434454679489136),
 ('далі', 0.5326734781265259),
 ('вулиця', 0.5264661312103271),
 ('менше', 0.5228954553604126)]

In [341]:
model6.wv.most_similar('паніка')

[('падіння', 0.6145803332328796),
 ('товар', 0.6104804277420044),
 ('черга', 0.5934183597564697),
 ('зазвичай', 0.5607894659042358),
 ('виробництво', 0.5506153106689453),
 ('нафта', 0.5496402978897095),
 ('ринка', 0.5485252737998962),
 ('грош', 0.543960452079773),
 ('супермаркет', 0.5425463318824768),
 ('попит', 0.5366764068603516)]

In [342]:
model6.wv.most_similar('пандемія')

[('китайські', 0.585578978061676),
 ('наслідок', 0.5708694458007812),
 ('країна', 0.5693174600601196),
 ('економічні', 0.560942530632019),
 ('італійські', 0.5522142648696899),
 ('спалах', 0.5394347310066223),
 ('італія', 0.5335023999214172),
 ('поширення', 0.5299360156059265),
 ('криз', 0.5227494835853577),
 ('здоров', 0.4960803985595703)]

In [360]:
model6.wv.most_similar('італія')

[('країна', 0.757164716720581),
 ('китай', 0.7312150597572327),
 ('найбільше', 0.729407548904419),
 ('найбільші', 0.7043057680130005),
 ('захворіймо', 0.6891632080078125),
 ('лідер', 0.674144983291626),
 ('інфіковані', 0.652220606803894),
 ('іспанія', 0.6483824253082275),
 ('регіон', 0.6347239017486572),
 ('стан_березень', 0.6339153051376343)]

In [361]:
model6.wv.most_similar('смерть')

[('помрімо', 0.8402688503265381),
 ('стан_березень', 0.8203840851783752),
 ('захворіймо', 0.7771463394165039),
 ('зараження', 0.7562247514724731),
 ('тис', 0.7258131504058838),
 ('захворювання', 0.7193724513053894),
 ('іспанія', 0.6894664764404297),
 ('офіційно', 0.6884250640869141),
 ('інфіковані', 0.680182158946991),
 ('хвороба', 0.6609140038490295)]

In [362]:
model6.wv.most_similar('обмеження')

[('режим', 0.6664921045303345),
 ('надзвичайні_стан', 0.6542678475379944),
 ('транспорт', 0.6445250511169434),
 ('автобус', 0.618003249168396),
 ('заборона', 0.6170927286148071),
 ('взагалі', 0.555680513381958),
 ('право', 0.5444482564926147),
 ('міжнародні', 0.5332208871841431),
 ('послуга', 0.525025486946106),
 ('закон', 0.5175245404243469)]